In [2]:
import sklearn
import sklearn.tree
import numpy as np
import graphviz

# Initiate classifier
infoGain_clf = sklearn.tree.DecisionTreeClassifier(criterion='entropy')


def load_data(type):

    col_names = [
        'RepubOrDemo', 'handicapped-infants', 'water-project-cost-sharing', 'adoption-of-the-budget-resolution',
        'physician-fee-freeze', 'el-salvador-aid', 'religious-groups-in-schools', 'anti-satellite-test-ban',
        'aid-to-nicaraguan-contras', 'mx-missile', 'immigration', 'synfuels-corporation-cutback', 'education-spending',
        'superfund-right-to-sue', 'crime', 'duty-free-exports', 'export-administration-act-south-africa'
    ]

    my_votes_array = np.genfromtxt('house-votes-84.data', dtype=None, delimiter=',')

    x_matrix = my_votes_array[:, 1:]
    feature_names = np.array(col_names[1:])

    y = my_votes_array[:, :1]

    label_name = ['Republican', 'Democrat']

    x_matrix = convert_to_zeros_and_ones(type, x_matrix)

    # Fit classifier
    infoGain_clf.fit(x_matrix, y)

    from StringIO import StringIO
    ss = StringIO()

    sklearn.tree.export_graphviz(infoGain_clf.tree_, feature_names=feature_names, class_names=label_name, 
                                 label="all", out_file=ss)
    graphviz.Source(ss.getvalue())

    get_prediction(-2, x_matrix, y)


def convert_to_zeros_and_ones(type, data):

    data[data == 'y'] = 1
    data[data == 'n'] = 0

    if type == 'ternary':
        data = convert_ternary(data)
    elif type == 'skip':
        data = skip(data)
    else:
        print 'finding common'

    data = data.astype(int)

    return data

def skip(data):
    


def convert_ternary(data):
    data[data == '?'] = 2

    return data


def get_prediction(row_num, x_matrix, y):
    print "predicted:", infoGain_clf.predict(x_matrix[-2:, :])
    print "truth", y[-2:]

load_data('ternary')

predicted: ['republican' 'republican']
truth [['republican']
 ['republican']]
